In [ ]:
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.runners.interactive.interactive_beam import *

from __future__ import absolute_import

import argparse
import logging
import re

from past.builtins import unicode

import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText

# An input file is downloaded through `gsutil cp gs://apache-beam-samples/shakespeare/kinglear.txt /tmp/`

In [ ]:
p = beam.Pipeline(InteractiveRunner())
lines = p | 'read' >> ReadFromText('/tmp/kinglear.txt')
show(lines)

In [ ]:
words = lines | 'split' >> beam.FlatMap(lambda line: re.findall(r'[\w\']+', line.strip(), re.UNICODE))
show(words)

In [ ]:
word_with_ones = words | 'pair_with_one' >> beam.Map(lambda word: (word, 1))
show(word_with_ones)

In [ ]:
ones_grouped_by_words = word_with_ones | 'group' >> beam.GroupByKey()
show(ones_grouped_by_words)

In [ ]:
counts = ones_grouped_by_words | 'count' >> beam.Map(lambda x: (x[0], sum(x[1])))
show(counts)

In [ ]:
output = counts | 'format' >> beam.Map(lambda x: '%s: %d' % (x[0], x[1]))
show(output)

In [ ]:
output | 'write' >> WriteToText('/tmp/output.txt')
p.run()